Comando abaixo apenas para que seja possível importar os modulos corretamente

In [1]:
import sys
sys.path.append("../") 

# Criação de uma Feature de Estrutura

As features de estrutura são features que calculam métricas baseadas na estrutura do texto. Em um texto HTML, por exemplo, essas métricas são calculadas com relevância em tags e texto puro. Algumas métricas estruturais são TagBasedFeature, baseando os cálculos no tagueamento do texto.

## TagBasedFeature

Uma TagBasedFeature considera tags inicial e final, além do conteúdo textual de um documento HTML. Neste projeto, utilizou-se a biblioteca HTMLParser do Python3, que é um parser para HTML simples e XHTML. Para saber mais sobre essa biblioteca e aprender sobre os métodos utilizados neste projeto, acesse: https://docs.python.org/3/library/html.parser.html.

### Entendendo o ParserTags

A biblioteca HTMLParser contém a interface ParserTags, que é a classe responsável por recolher e classificar as tags de um documento HTML. No bloco abaixo exemplifica-se a importação da biblioteca e o uso dos métodos. A implementação dos métodos foi abstraída.

In [2]:
from html.parser import HTMLParser

class ParserTags(HTMLParser):
    def __init__(self, arrParserFeats, document):
        super(HTMLParser,self).__init__(self)
    
    def handle_data(self,str_data):
        pass

    def handle_starttag(self, tag, attrs):
        pass
        
    def handle_endtag(self, tag):
        pass

A implementação do ParserTags pode ser vista no módulo feature.features. Ela foi recriada de modo a tentar diminuir o custo de processamento, já que várias features de estrutura podem ser executadas por documento. A lógica foi separar o processamento pelos tipo de tag: inicial (ex. &lt;p>), final (ex. &lt;/p>) e os dados puros do HTML (ex. o texto contido em um parágrafo). Essa separação ocorre de modo a criar diferentes listas de features a serem executadas a partir da lista inicial, que contém todas as requisições de TagBasedFeature. Abaixo, um recorte da implementação contida no módulo features.

In [3]:
class ParserTags(HTMLParser):
    
    def handle_data(self,str_data):
            if(self.firstTimeData):
                self.arrFeatsData = [feat for feat in self.arrParserFeats if feat.data(self.document, str_data)]
                self.firstTimeData = False
            else:
                for feat in self.arrFeatsData:
                    feat.data(self.document, str_data)

O código contém booleanos que marcam a primeira execução de cada método do ParserTags. Quando a primeira execução é realizada, a lista de features a serem processadas é dividida nas categorias citadas, ou seja, em três outras listas. Essa divisão é feita pelo "for pythonico" que pode ser observado no código acima, onde um método TagBasedFeature, que será  exemplificado posteriormente, é chamado na primeira execução. Caso a execução não for a primeira e a divisão das listas já tiverem sido feitas, o respectivo método TagBasedFeature é chamado novamente.

A implementação aplicada ao contexto do método handle_data é aplicada ao contexto dos outros métodos dessa classe, cada um com seu respectivo método TagBased.

O planejamento apresentado garante que o parser execute somente uma vez por documento, e não a cada vez que uma TagBasedFeature é executada, já que um documento pode requisitar várias features desse tipo.

### Entendendo o TagBasedFeature

Uma feature TagBased leva em consideração as tags presentes em um documento HTML. as tags HTML podem ser divididas em tags iniciais e finais. Podemos considerar o conteúdo textual presente, que chamamos de dados. Vejamos o exemplo da seção "form":

```<form action="example.php" method="get">
</form>```

Vimos que a seção possui uma tag inicial com dois atributos (action e method) e uma tag final. Uma TagBasedFeature considera todos esses dados, incluindo atributos. Inicialmente, os 3 métodos dessa classe retornam o valor falso, retornando verdadeiro de acordo com o tipo de tag utilizada na feature implementada. O objetivo é diminuir a alocação de métodos que não estão sendo utilizados, como mencionado anteriormente na classe ParserTags.

## Implementando uma TagBasedFeature

Com todas as questões já discutidas, façamos a implementação de uma TagBasedFeature de exemplo. A classe FormRatio calcula a proporção entre métodos get e post de formulários. Para isso, precisamos utilizar a tag form inicial e seus atributos para verificarmos qual tipo de método está sendo utilizado na submissão. Abaixo temos a implementação da feature de exemplo.

In [4]:
from feature.features import TagBasedFeature

class FormRatio(TagBasedFeature):
    def __init__(self,name,description,reference,visibility,text_format,feature_time_per_document):
        super(TagBasedFeature,self).__init__(name,description,reference,visibility,text_format,feature_time_per_document)
        self.int_get_method = 0
        self.int_post_method = 0
    
    def isFormTag(tag):
        if tag is "form":
            return True
        return False
    
    def startTag(self,document,tag,attrs):
        if(self.isFormTag(tag)):
            for item in attrs:
                if item[0] is "method" and item[1] is "get":
                    self.int_get_method = self.int_get_method + 1
                if item[0] is "method" and item[1] is "post":
                    self.int_post_method = self.int_post_method + 1
        super.startTag(document,tag,attrs)
        return True
    
    def compute_feature(self,document):
        return self.int_get_method/self.int_post_method
    
    def finish_document(self,document):
        self.int_get_method = 0
        self.int_post_method = 0

A classe acima verifica se a tag é um formulário, e após isso conta quantos métodos get e post possuem no documento. Com esses dados recolhidos, calcula a razão entre a ocorrência dos dois métodos.

## Testando a feature implementada

Para fazer o teste manual da classe implementada, executamos seus métodos nas suas respectivas funções. Abaixo um código de teste para a execução da feature, em teste unitário.

In [6]:
import unittest
from feature.features import FeatureVisibilityEnum, Document, FeatureCalculator
from utils.basic_entities import FormatEnum, FeatureTimePerDocumentEnum

class TestTagBased(unittest.TestCase):
    
    def testParser(self):
        tcount = FormRatio("FormRatio", "Calcula a razão entre métodos get e post", "Jupyter-notebook documentation", 
                                         FeatureVisibilityEnum.public, 
                                         FormatEnum.HTML, 
                                         FeatureTimePerDocumentEnum.MILLISECONDS)
        document = Document(1,"doc1",'<div><form action="example.php" method="post"></form><form method="get"></form></div>')
        
        tcount.feed("<head></head><body>Dados<p>Parágrafo</p><h1><h123> de teste</body>")
        int_result = tcount.compute_feature(document)
        self.assertEqual(int_result, 1, "O teste deu errado")    

sys.argv = ['', 'TestTagBased.testParser']
unittest.main()

E
ERROR: testParser (__main__.TestTagBased)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-6-f84a4be92e24>", line 14, in testParser
    tcount.feed("<head></head><body>Dados<p>Parágrafo</p><h1><h123> de teste</body>")
AttributeError: 'FormRatio' object has no attribute 'feed'

----------------------------------------------------------------------
Ran 1 test in 0.002s

FAILED (errors=1)


SystemExit: True

/usr/lib/python3/dist-packages/IPython/core/interactiveshell.py:2889: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
